# 得られたデータセットを時間で区切って新しいcsvを作る
基本的に時間軸はscrapingで決められているので、やることとしては、与えられた横軸を指定の個数で区切ることをする

In [1]:
import pandas as pd

import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import seaborn as sns
import sys
import stockpy as st
from datetime import datetime

In [2]:
df = pd.read_csv('../database/normaraze_stock_price/dataset_20231026.csv', header=0)
df

,name,2023-07-26,2023-07-27,2023-07-28,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
0,AACI,10.570,10.5700,10.590000,10.570,10.500,10.5600,10.570,10.500,10.5000,...,10.630,10.69,10.720,10.690,10.690,10.690,10.690,10.690,10.690,NaN
1,AADI,5.760,5.3800,5.360000,5.330,5.250,5.2600,5.660,5.730,5.9900,...,4.200,4.23,4.220,4.180,3.990,3.960,4.090,3.940,4.290,4.2200
2,AAL,16.760,16.5700,16.790001,16.750,16.230,15.9500,15.980,15.840,15.9900,...,12.060,11.72,11.940,11.940,11.360,11.450,11.080,11.290,11.210,11.0400
3,AAME,1.940,1.8700,1.830000,1.830,1.770,1.7600,1.810,1.920,1.8600,...,1.860,1.85,1.820,1.730,1.800,1.930,1.810,1.940,1.940,1.8000
4,AAOI,7.950,6.1900,6.710000,6.750,7.030,7.3900,6.590,11.010,12.3800,...,7.950,7.55,7.650,7.590,7.550,7.450,6.910,7.170,7.440,7.1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,ZVRA,4.710,4.6400,4.830000,4.880,4.900,4.8100,4.840,4.820,4.8600,...,4.560,4.47,4.460,4.530,4.370,4.370,4.210,4.290,4.370,4.3900
2948,ZVSA,0.166,0.1620,0.169000,0.173,0.170,0.1700,0.160,0.170,0.1640,...,0.131,0.13,0.126,0.122,0.128,0.128,0.125,0.118,0.116,0.1145
2949,ZYME,7.510,7.2800,7.270000,7.460,7.450,7.3200,7.320,7.390,7.2600,...,6.420,6.50,7.050,7.250,7.160,7.210,7.190,7.000,7.010,6.9700
2950,ZYNE,0.364,0.3551,0.359800,0.365,0.355,0.3688,0.352,0.353,0.3421,...,1.300,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from sklearn.preprocessing import LabelEncoder

# ラベルを数値に変換する
le = LabelEncoder()
df['encode_name'] = le.fit_transform(df['name'])
df.insert(1, 'encode_name', df.pop('encode_name'))

In [4]:
x = df.iloc[:, 2:]
label = df.iloc[:, :2]

In [5]:
x

,2023-07-26,2023-07-27,2023-07-28,2023-07-31,2023-08-01,2023-08-02,2023-08-03,2023-08-04,2023-08-07,2023-08-08,...,2023-10-12,2023-10-13,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-23,2023-10-24,2023-10-25
0,10.570,10.5700,10.590000,10.570,10.500,10.5600,10.570,10.500,10.5000,10.5000,...,10.630,10.69,10.720,10.690,10.690,10.690,10.690,10.690,10.690,NaN
1,5.760,5.3800,5.360000,5.330,5.250,5.2600,5.660,5.730,5.9900,5.8400,...,4.200,4.23,4.220,4.180,3.990,3.960,4.090,3.940,4.290,4.2200
2,16.760,16.5700,16.790001,16.750,16.230,15.9500,15.980,15.840,15.9900,16.1800,...,12.060,11.72,11.940,11.940,11.360,11.450,11.080,11.290,11.210,11.0400
3,1.940,1.8700,1.830000,1.830,1.770,1.7600,1.810,1.920,1.8600,1.8200,...,1.860,1.85,1.820,1.730,1.800,1.930,1.810,1.940,1.940,1.8000
4,7.950,6.1900,6.710000,6.750,7.030,7.3900,6.590,11.010,12.3800,12.1900,...,7.950,7.55,7.650,7.590,7.550,7.450,6.910,7.170,7.440,7.1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,4.710,4.6400,4.830000,4.880,4.900,4.8100,4.840,4.820,4.8600,5.2000,...,4.560,4.47,4.460,4.530,4.370,4.370,4.210,4.290,4.370,4.3900
2948,0.166,0.1620,0.169000,0.173,0.170,0.1700,0.160,0.170,0.1640,0.1610,...,0.131,0.13,0.126,0.122,0.128,0.128,0.125,0.118,0.116,0.1145
2949,7.510,7.2800,7.270000,7.460,7.450,7.3200,7.320,7.390,7.2600,7.2900,...,6.420,6.50,7.050,7.250,7.160,7.210,7.190,7.000,7.010,6.9700
2950,0.364,0.3551,0.359800,0.365,0.355,0.3688,0.352,0.353,0.3421,0.3463,...,1.300,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_data = x

In [ ]:
for data_label, data in enumerate([train_data]):
    start = 0
    length = 60
    df_list = []
    for i in range(data.shape[1]):
        if i < data.shape[1] - length:
            new_x = data.iloc[:, start+i:length+i]
            df_combined = pd.concat([label, new_x], axis=1, ignore_index=True)
            df_list.append(df_combined)
    # リストに格納されたデータフレームを行方向に結合する
    df_merged = pd.concat(df_list, axis=0, ignore_index=True)
    # Reset the index
    df_merged.reset_index(drop=True, inplace=True)
    # データフレームをCSVファイルとして保存する
    df_merged.to_csv('train_dataset_by_time2.csv', index=False)

In [29]:
train_data = x.iloc[:, :i]
valid_data = x.iloc[:, i:]

In [30]:
new_df = pd.DataFrame()

In [31]:
for data_label, data in enumerate([train_data, valid_data]):
    print(data_label)
    print(data)

0
      2021-10-21  2021-10-22  2021-10-25  2021-10-26  2021-10-27  2021-10-28  \
0       0.943429    0.951862    0.951511    0.945538    0.949754    0.961349   
1       0.893531    0.860287    0.863881    0.871069    0.854897    0.862084   
2       0.929047    0.906874    0.922395    0.904656    0.882483    0.911308   
3       0.485788    0.484496    0.490310    0.484496    0.496770    0.494832   
4       0.648051    0.652387    0.647682    0.645837    0.638273    0.659767   
...          ...         ...         ...         ...         ...         ...   
2676    0.635088    0.626316    0.671930    0.671930    0.687719    0.719298   
2677    0.754164    0.751236    0.758191    0.742449    0.728171    0.735127   
2678    0.949318    0.983431    0.963938    1.000000    0.984405    0.979532   
2679    0.945366    0.930797    0.960340    0.938891    0.917038    0.924727   
2680    0.659891    0.659355    0.665242    0.662031    0.645976    0.661496   

      2021-10-29  2021-11-01  2021-11

In [32]:
valid_data.shape

(2681, 65)

In [33]:
for data_label, data in enumerate([train_data, valid_data]):
    start = 0
    length = 60
    df_list = []
    for i in range(data.shape[1]):
        if i < data.shape[1] - length:
            new_x = data.iloc[:, start+i:length+i]
            df_combined = pd.concat([label, new_x], axis=1, ignore_index=True)
            df_list.append(df_combined)
    # リストに格納されたデータフレームを行方向に結合する
    df_merged = pd.concat(df_list, axis=0, ignore_index=True)
    # Reset the index
    df_merged.reset_index(drop=True, inplace=True)

    if data_label == 0:
        df_merged.to_csv('train_dataset_by_time.csv', index=False)
    else:
        df_merged.to_csv('valid_dataset_by_time.csv', index=False)

In [58]:
df_merged

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,AADI,0.432537,0.445186,0.439916,0.438862,0.426212,0.432186,0.431834,0.456781,0.455025,...,0.272663,0.264231,0.266690,0.270555,0.253338,0.256149,0.254744,0.254744,0.240689,0.250176
1,AAL,0.572327,0.610512,0.628482,0.637017,0.656334,0.682390,0.689128,0.756065,0.764600,...,0.634322,0.628032,0.627134,0.645553,0.620845,0.612309,0.616352,0.626235,0.628032,0.644654
2,AAME,0.539550,0.541779,0.550698,0.544009,0.575223,0.572993,0.557386,0.555157,0.559616,...,0.557386,0.555157,0.561845,0.552927,0.537320,0.555157,0.555157,0.552927,0.557386,0.557386
3,AAOI,0.121447,0.124031,0.121447,0.120155,0.138889,0.136951,0.138889,0.151809,0.156977,...,0.160207,0.152455,0.155039,0.156977,0.153101,0.158915,0.155685,0.111111,0.117571,0.118217
4,AAON,0.699674,0.670127,0.659187,0.693689,0.695747,0.706967,0.710333,0.717532,0.728098,...,0.859744,0.836885,0.841195,0.858808,0.856278,0.865178,0.854873,0.869488,0.889817,0.887944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357610,ZUMZ,0.334066,0.339374,0.332784,0.329489,0.341571,0.339191,0.344865,0.343401,0.346330,...,0.318689,0.323815,0.325828,0.316859,0.312283,0.318872,0.308805,0.307889,0.307706,0.312649
357611,ZVRA,0.475634,0.481481,0.468811,0.475634,0.464912,0.464912,0.459064,0.452242,0.470760,...,0.479532,0.482943,0.469786,0.460039,0.448343,0.447368,0.447368,0.458090,0.450292,0.462963
357612,ZYME,0.303007,0.303392,0.299537,0.303392,0.284888,0.287201,0.289514,0.280648,0.280262,...,0.239784,0.240941,0.244410,0.238628,0.238242,0.239784,0.241326,0.246723,0.242097,0.248651
357613,ZYNE,0.084884,0.086279,0.083721,0.083721,0.083721,0.082791,0.084651,0.082558,0.083721,...,0.295349,0.297674,0.296512,0.295349,0.293023,0.281395,0.290698,0.297674,0.295349,0.302326


In [59]:
df_merged.to_csv('train_dataset_by_time2.csv', index=False)

In [34]:
for data_label, data in enumerate([train_data, valid_data]):
    for i in range(data.shape[1]):
        if i < data.shape[1] - length:
            new_x = data.iloc[:, start+i:length+i]
            df_combined = pd.concat([label, new_x], axis=1, ignore_index=True)
            df_list.append(df_combined)  # リストに追加
            # if 495 < i:
            #     print(data.iloc[:, start+i:length+i])
    # リストに格納されたデータフレームを行方向に結合する
    df_merged = pd.concat(df_list, axis=0, ignore_index=True)

    # Reset the index
    df_merged.reset_index(drop=True, inplace=True)

    if data_label == 0:
        df_merged.to_csv('train_dataset_by_time.csv', index=False)
    else:
        df_merged.to_csv('valid_dataset_by_time.csv', index=False)


KeyboardInterrupt: 

ここから学習用のやつ作成

In [226]:
x = df_merged.iloc[:, 1:]
label = df_merged.iloc[:, :1]

In [227]:
x

,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,...,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858,0.848559
1,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,...,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770,0.806828
2,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,...,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408,0.543237
3,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,...,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607,0.332041
4,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,...,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002,0.735409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,0.550388,0.544574,...,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667,0.575581
3532,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,0.956355,0.949597,...,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651,0.792787
3533,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,0.981166,0.985594,...,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762,0.885185
3534,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,0.755459,0.751576,...,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656,0.694269


In [228]:
x = x.values.tolist()

In [229]:
def process_data(data):
    try:
        if ((0.01 < data[-1] - data[-2])):
            data.pop()
            data.insert(0, 1)
        else:
            data.pop()
            data.insert(0, 0)
        # # リストをNumPy配列に変換
        # data = np.array([data])
        return data
    except Exception as e:
        print("Error processing data")
        print(e)
        raise Exception

In [230]:
data_list = []
for i in x:
    data_list.append(process_data(i))

In [231]:
df_merged_2 = pd.DataFrame(data_list)

In [232]:
df_combined_2 = pd.concat([label, df_merged_2], axis=1, ignore_index=True)

In [233]:
df_combined_2

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,AADI,0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,AAL,0,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,AAME,0,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,AAOI,0,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,AAON,0,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,AAOI,0,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,...,0.635659,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667
3532,AAON,1,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,...,0.862988,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651
3533,AAPL,0,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,...,0.894513,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762
3534,ABCB,0,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,...,0.705712,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656


In [241]:
df_combined_2.to_csv('dataset_for_model2.csv', index=False)

In [242]:
# カテゴリごとのカウントを取得
value_counts = df_combined_2[1].value_counts()

# 最も少ないカウント数を取得
min_count = value_counts.min()

# 最も少ないカウント数のカテゴリを取得
min_count_categories = value_counts[value_counts == min_count].index

# フィルタリング
filtered_df_1 = df_combined_2[df_combined_2[1].isin(min_count_categories)]

In [243]:
# カテゴリごとのカウントを取得
value_counts = df_combined_2[1].value_counts()

# 最も少ないカウント数を取得
max_count = value_counts.max()

# 最も少ないカウント数のカテゴリを取得
max_count_categories = value_counts[value_counts == max_count].index

# フィルタリング
filtered_df_2 = df_combined_2[df_combined_2[1].isin(max_count_categories)][:min_count]

In [246]:
filtered_df_2

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,AADI,0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,AAL,0,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,AAME,0,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,AAOI,0,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,AAON,0,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,ABCB,0,0.749487,0.740048,0.739685,0.768909,0.775807,0.770361,0.748216,0.748035,...,0.748398,0.747309,0.741319,0.732392,0.732027,0.744051,0.751157,0.750428,0.751521,0.750975
1047,ABCL,0,0.508178,0.507050,0.474337,0.489002,0.477157,0.460801,0.450085,0.454597,...,0.600677,0.579244,0.578680,0.570220,0.600677,0.600113,0.585448,0.596729,0.648054,0.653130
1048,AADI,0,0.580112,0.546381,0.569923,0.575896,0.549192,0.539354,0.571328,0.544975,...,0.425158,0.418833,0.456079,0.432888,0.448700,0.471890,0.472593,0.481377,0.484540,0.467674
1050,AAME,0,0.764734,0.706766,0.702307,0.700077,0.702307,0.713454,0.706766,0.702307,...,0.624273,0.601977,0.584141,0.595289,0.566304,0.572993,0.608666,0.675552,0.682241,0.697848


In [247]:
result = pd.concat([filtered_df_1, filtered_df_2], axis=0)
result.to_csv('dataset_for_model3.csv', index=False)